## Сбор и расчет других признаков

In [10]:
import pandas as pd
import requests, time, io
from math import isnan, cos, radians
import numpy as np
import osmnx as ox, networkx as nx

In [11]:
pora10_ind = pd.read_excel('data/made/pora10_ind.xlsx')
pora10_ind.columns

Index(['territory_id', 'settlement_name_sep', 'municipality_up_name_actual',
       'region', 'municipality_up_name', 'municipality_down_name',
       'settlement_name', 'type', 'arctic', 'remote',
       ...
       'Сельское хозяйство', 'Спорт и досуг', 'Строительство', 'Торговля',
       'Транспортировка и хранение', 'Услуги ЖКХ', 'Финансы и страхование',
       'min_salary', 'max_salary', 'rub_for_life'],
      dtype='object', length=130)

### Вакансии на hhru по населенным пунктам арктической зоны
- Срез на 07.11.2025 (собрано вручную)

In [12]:
vacancies = pd.read_excel('data/made/hhru_vacancies_07-11-2025.xlsx')

In [13]:
print(vacancies.shape)
vacancies.head()

(46, 2)


,settlement_name_sep,hhru_vacancies_count
0,населенный пункт Видяево (Мурманская область),7
1,пгт Березово (Ханты-Мансийский автономный окру...,3
2,пгт Верхнетуломский (Мурманская область),1
3,пгт Воргашор (Республика Коми),0
4,пгт Депутатский (Республика Саха (Якутия)),0


In [14]:
pora10_ind = pd.merge(pora10_ind, vacancies, on = 'settlement_name_sep', how = 'inner')

### OSM. POI
подсчет POI в радиусе 7 км вокруг координат населенного пункта
- `amenity`
- `shop`
- `tourism`
- `leisure`
- `office`
- `craft`
- `man_made`
- `historic`
- `aeroway`
- `healthcare`
- `education`
- `sport`
- `public_transport`

In [15]:
# тэги
TAG_KEYS = [
    "amenity","shop","tourism","leisure","office","craft","man_made",
    "historic","aeroway","healthcare","education","sport","public_transport"
]

OVERPASS_URLS = [
    "https://overpass-api.de/api/interpreter",
    "https://overpass.openstreetmap.ru/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
]

HEADERS = {
    "User-Agent": "OSM-POI-count/1.0 (contact: your_email@example.com)"
}

def count_poi_overpass(lat: float, lon: float, radius_m: int = 7000, max_attempts: int = 6, pause_base: float = 1.0):
    key_regex = "|".join(TAG_KEYS)
    query = f"""
    [out:json][timeout:90];
    (
      node(around:{radius_m},{lat},{lon})[~"^({key_regex})$"~".+"];
      way(around:{radius_m},{lat},{lon})[~"^({key_regex})$"~".+"];
      relation(around:{radius_m},{lat},{lon})[~"^({key_regex})$"~".+"];
    );
    out ids;
    """

    for attempt in range(max_attempts):
        for url in OVERPASS_URLS:
            try:
                resp = requests.post(url, data={"data": query}, headers=HEADERS, timeout=120)
                if resp.status_code in (429, 504, 502, 503, 500):
                    time.sleep(pause_base * (2 ** attempt))
                    continue
                resp.raise_for_status()
                data = resp.json()
                els = data.get("elements", [])
                uniq = {f"{el.get('type','?')[0]}{el.get('id','')}" for el in els}
                return len(uniq)
            except requests.RequestException:
                time.sleep(pause_base * (2 ** attempt))
                continue
    return pd.NA

poi_counts = []
for lat, lon in zip(pora10_ind["latitude"], pora10_ind["longitude"]):
    if pd.isna(lat) or pd.isna(lon):
        poi_counts.append(pd.NA)
        continue
    cnt = count_poi_overpass(float(lat), float(lon), radius_m=7000)
    poi_counts.append(cnt)
    time.sleep(1.0)

pora10_ind["POI_num"] = poi_counts
pora10_ind[["latitude","longitude","POI_num"]].head()

,latitude,longitude,POI_num
0,64.897914,45.758867,441
1,63.707442,37.453802,30
2,64.000907,44.449961,112
3,64.475700,40.862821,789
4,64.389661,40.631610,678


### OSM. Количество жилых домов
подсчет жилых домов в радиусе 7 км вокруг координат населенного пункта

building:
- `house`
- `residential`
- `apartments`
- `flats`
- `detached`
- `terrace`
- `semidetached_house`
- `dormitory`
- `villa`
- `bungalow`
- `static_caravan`

In [19]:
RESI_RE = "^(house|residential|apartments|flats|detached|terrace|semidetached_house|dormitory|villa|bungalow|static_caravan)$"

OVERPASS_URLS = [
    "https://overpass-api.de/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
    "https://overpass.openstreetmap.ru/api/interpreter",
]

HEADERS = {"User-Agent": "OSM-Residential-Counter/1.0 (contact: your_email@example.com)"}

def count_residential_overpass(lat: float, lon: float, radius_m: int = 7000,
                               max_attempts: int = 6, pause_base: float = 1.0):
    q = f"""
    [out:json][timeout:90];
    (
      way(around:{radius_m},{lat},{lon})["building"~"{RESI_RE}"];
      relation(around:{radius_m},{lat},{lon})["building"~"{RESI_RE}"];
      way(around:{radius_m},{lat},{lon})["building:use"~"residential"];
      relation(around:{radius_m},{lat},{lon})["building:use"~"residential"];
      node(around:{radius_m},{lat},{lon})["building"~"{RESI_RE}"];
    );
    out ids;
    """
    for attempt in range(max_attempts):
        for url in OVERPASS_URLS:
            try:
                r = requests.post(url, data={"data": q}, headers=HEADERS, timeout=120)
                if r.status_code in (429, 500, 502, 503, 504):
                    time.sleep(pause_base * (2 ** attempt))
                    continue
                r.raise_for_status()
                data = r.json()
                els = data.get("elements", [])
                uniq = {f"{e.get('type','?')[0]}{e.get('id','')}" for e in els}
                return len(uniq)
            except requests.RequestException:
                time.sleep(pause_base * (2 ** attempt))
                continue
    return pd.NA

building_counts = []
for lat, lon in zip(pora10_ind["latitude"], pora10_ind["longitude"]):
    if pd.isna(lat) or pd.isna(lon):
        building_counts.append(pd.NA)
    else:
        building_counts.append(count_residential_overpass(float(lat), float(lon), radius_m=7000))
        time.sleep(1.0)  # щадим Overpass

pora10_ind["building_num"] = building_counts
pora10_ind[["settlement_name_sep","building_num"]].head()

,settlement_name_sep,building_num
0,село Лешуконское (Архангельская область),638
1,рп Малошуйка (Архангельская область),16
2,село Карпогоры (Архангельская область),191
3,поселок Уемский (Архангельская область),224
4,посёлок Катунино (Архангельская область),53


### Расстояние до ближайшего аэропорта по дорожному графу OSM
- Аэропорты выгружаем вот [отсюда](https://ourairports.com/data), файл - `airports.csv`

In [20]:
NETWORK_TYPE = "drive"
R_GRAPH_M    = 400_000
NEAR_KM      = 1.5 * (R_GRAPH_M/1000)
FALLBACK_KM  = 300
MAX_CAND     = 8
MIN_BUF_KM   = 15
MAX_BUF_KM   = 60

OVERPASS_URLS = [
    "https://overpass.kumi.systems/api/interpreter",
    "https://lz4.overpass-api.de/api/interpreter",
    "https://overpass-api.de/api/interpreter",
    "https://z.overpass-api.de/api/interpreter",
    "https://overpass.openstreetmap.ru/api/interpreter",
]

ox.settings.use_cache = False
ox.settings.log_console = False
ox.settings.requests_timeout = 300
ox.settings.request_config = {"headers": {"User-Agent": "airport-road-distance/1.0 (contact: your_email@example.com)"}}

def load_airports():
    url = "https://ourairports.com/data/airports.csv"
    for i in range(4):
        try:
            r = requests.get(url, timeout=60)
            r.raise_for_status()
            df_air = pd.read_csv(io.StringIO(r.text))
            df_air = df_air[df_air["type"].isin(["small_airport","medium_airport","large_airport"])]
            df_air = df_air[["name","type","scheduled_service",
                             "latitude_deg","longitude_deg"]].dropna(subset=["name",
                                                                             "type",
                                                                             "latitude_deg",
                                                                             "longitude_deg"])
            return df_air.reset_index(drop=True)
        except Exception:
            time.sleep(2**i)
    return pd.DataFrame(columns=["name","type","scheduled_service","latitude_deg","longitude_deg"])

air = load_airports()

# примерный фильтр по арктической зоне РФ, чтобы не нагружать алгоритм всеми аэропортами мира
air = air[(air['latitude_deg']>62)&((air['longitude_deg']<-171)|(air['longitude_deg']>30))]

def haversine_km(lat, lon, lats, lons):
    R = 6371.0088
    lat1 = np.radians(lat); lon1 = np.radians(lon)
    lat2 = np.radians(lats); lon2 = np.radians(lons)
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

def bbox_covering_pair(lat1, lon1, lat2, lon2, buf_km):
    lat_c = (lat1 + lat2) / 2.0
    m_per_deg_lat = 111_320.0
    m_per_deg_lon = 111_320.0 * max(1e-6, cos(radians(lat_c)))
    dlat = (buf_km * 1000.0) / m_per_deg_lat
    dlon = (buf_km * 1000.0) / m_per_deg_lon
    north = min(90.0,  max(lat1, lat2) + dlat)
    south = max(-90.0, min(lat1, lat2) - dlat)
    east  = min(180.0, max(lon1, lon2) + dlon)
    west  = max(-180.0, min(lon1, lon2) - dlon)
    return north, south, east, west

def graph_covering_pair_retry(lat1, lon1, lat2, lon2, buf_km):
    north, south, east, west = bbox_covering_pair(lat1, lon1, lat2, lon2, buf_km)
    last_exc = None
    for url in OVERPASS_URLS:
        try:
            ox.settings.overpass_url = url
            return ox.graph.graph_from_bbox(
                north, south, east, west,
                network_type=NETWORK_TYPE, simplify=True
            )
        except Exception as e:
            last_exc = e
            time.sleep(1.0)
    raise last_exc

if air.empty:
    pora10_ind["dist_to_airport_osm_m"]  = np.nan
    pora10_ind["dist_to_airport_geo_km"] = np.nan
else:
    air_lat = air["latitude_deg"].to_numpy(dtype=float)
    air_lon = air["longitude_deg"].to_numpy(dtype=float)

    road_m_list, geo_km_list = [], []
    road_air_name, geo_air_name = [], []

    for lat, lon in zip(pora10_ind["latitude"].astype(float), pora10_ind["longitude"].astype(float)):
        try:
            approx = haversine_km(lat, lon, air_lat, air_lon)

            ig = int(np.argmin(approx))
            min_geo_km = float(approx[ig])
            geo_km_list.append(min_geo_km)
            geo_air_name.append(str(air.iloc[ig]["name"]))

            idx = np.where(approx <= NEAR_KM)[0]
            if idx.size == 0:
                idx = np.where(approx <= FALLBACK_KM)[0]
            if idx.size == 0:
                road_m_list.append(np.nan)
                road_air_name.append(None)
                print(f'({lat:.4f}, {lon:.4f}): geo {min_geo_km:.1f} km, road None')
                continue

            cand_idx = idx[np.argsort(approx[idx])[:MAX_CAND]]
            cand = air.iloc[cand_idx].reset_index(drop=True)

            best_m = np.inf
            best_name = None

            for _, row in cand.iterrows():
                alat, alon = float(row["latitude_deg"]), float(row["longitude_deg"])
                dist_km = float(haversine_km(lat, lon, np.array([alat]), np.array([alon]))[0])
                buf_km = float(np.clip(10 + 0.05*dist_km, MIN_BUF_KM, MAX_BUF_KM))

                try:
                    G = graph_covering_pair_retry(lat, lon, alat, alon, buf_km=buf_km)
                except Exception:
                    continue

                try:
                    Gu = ox.utils_graph.get_undirected(G)
                    u = ox.distance.nearest_nodes(Gu, X=[lon], Y=[lat])[0]
                    v = ox.distance.nearest_nodes(Gu, X=[alon], Y=[alat])[0]
                    d_m = nx.shortest_path_length(Gu, u, v, weight="length")
                    if d_m < best_m:
                        best_m = d_m
                        best_name = str(row["name"])
                except Exception:
                    pass

                time.sleep(0.1)

            road_m_list.append(best_m if np.isfinite(best_m) else np.nan)
            road_air_name.append(best_name)
            print(f'({lat:.4f}, {lon:.4f}): geo {min_geo_km:.1f} km, road {best_m if np.isfinite(best_m) else None}')
        except Exception:
            geo_km_list.append(np.nan)
            road_m_list.append(np.nan)
            road_air_name.append(None)
            geo_air_name.append(None)

    pora10_ind["dist_to_airport_osm_m"]   = road_m_list
    pora10_ind["dist_to_airport_geo_km"]  = geo_km_list
    pora10_ind["airport_geo_nearest"]     = geo_air_name
    pora10_ind["airport_osm_nearest"]     = road_air_name

pora10_ind[["latitude","longitude","dist_to_airport_geo_km","dist_to_airport_osm_m","airport_geo_nearest","airport_osm_nearest"]].head()

(64.8979, 45.7589): geo 1.7 km, road None
(63.7074, 37.4538): geo 39.7 km, road None
(64.0009, 44.4500): geo 1.4 km, road None
(64.4757, 40.8628): geo 15.5 km, road None
(64.3897, 40.6316): geo 11.6 km, road None
(71.9803, 102.4762): geo 0.6 km, road None
(65.7940, 87.9555): geo 1.0 km, road None
(64.2743, 100.2108): geo 12.6 km, road None
(69.0563, 33.2923): geo 5.7 km, road None
(69.3195, 32.7996): geo 40.3 km, road None
(66.9630, 30.3473): geo 150.1 km, road None
(66.8478, 32.3698): geo 86.2 km, road None
(68.8157, 32.8099): geo 1.0 km, road None
(68.8527, 33.0175): geo 9.9 km, road None
(68.8044, 33.1017): geo 11.8 km, road None
(68.6073, 31.7986): geo 43.1 km, road None
(67.9423, 34.5538): geo 20.6 km, road None
(69.4108, 30.2143): geo 42.4 km, road None
(69.5462, 31.2099): geo 32.7 km, road None
(66.6848, 34.3439): geo 2.3 km, road None
(67.6768, 53.1293): geo 4.1 km, road None
(65.1990, 31.1870): geo 2.7 km, road None
(66.0741, 33.0448): geo 91.5 km, road None
(63.8910, 34.2612)

,latitude,longitude,dist_to_airport_geo_km,dist_to_airport_osm_m,airport_geo_nearest,airport_osm_nearest
0,64.897914,45.758867,1.705299,NaN,Leshukonskoye Airport,None
1,63.707442,37.453802,39.735806,NaN,Onega Airfield,None
2,64.000907,44.449961,1.366938,NaN,Karpogory Airfield,None
3,64.475700,40.862821,15.516483,NaN,Talagi Airport,None
4,64.389661,40.631610,11.622621,NaN,Vaskovo Airport,None


In [21]:
pora10_ind = pora10_ind.drop(columns = ['dist_to_airport_osm_m', 'airport_osm_nearest'], axis = 1)

pora10_ind = pd.merge(pora10_ind,
air[['name', 'type', 'scheduled_service']],
suffixes = ('_settlement', '_airport'),
left_on = "airport_geo_nearest",
right_on = 'name', how = 'left')

In [142]:
# pora10_ind = pora10_ind.rename(columns = {'type_airport': 'type'})
# pora10_ind = pora10_ind[list(pora10_ind.columns)[:-3]]

In [24]:
pora10_ind['scheduled_service'].value_counts()

scheduled_service
no     27
yes    19
Name: count, dtype: int64

In [25]:
pora10_ind['type_airport'].value_counts()

type_airport
small_airport     26
medium_airport    20
Name: count, dtype: int64

In [26]:
other_features = pora10_ind[['settlement_name_sep', 'latitude', 'longitude', 'hhru_vacancies_count',
                             'POI_num', 'building_num','dist_to_airport_geo_km', 'airport_geo_nearest',
                             'name', 'type_airport','scheduled_service', 'rub_for_life']]

### Циан
- срез объявлений о продаже в рассматриваемых арктических населенных пунктах на 08.11.2025

In [27]:
cian = pd.read_excel('data/made/cian_settlements_08_11_2025.xlsx').dropna(subset='cost_mln_rub')

In [28]:
cian.head()

,settlement_name_sep,cost_mln_rub,S_m2,rooms
0,пгт Сафоново (Мурманская область),2.25,32.4,1.0
1,пгт Сафоново (Мурманская область),1.70,31.0,1.0
2,пгт Сафоново (Мурманская область),2.00,43.9,2.0
3,пгт Сафоново (Мурманская область),2.00,45.9,2.0
4,пгт Сафоново (Мурманская область),2.10,45.9,2.0


In [29]:
cian['1000rub_per_m2'] = round(cian['cost_mln_rub']/ cian['S_m2'] * 1000, 2)
house_data = round(cian.groupby('settlement_name_sep')['1000rub_per_m2'].agg(['mean', 'count']),2).reset_index()
house_data = house_data.rename(columns={'count': 'sale_house_count', 'mean': '1000rub_per_m2_avg'})

other_features = pd.merge(other_features, house_data, on = 'settlement_name_sep', how = 'left').drop(columns ='name', axis = 1)
other_features['sale_house_count'] = other_features['sale_house_count'].fillna(0)
other_features.shape

(46, 13)

### Год основания

In [30]:
set_year = pd.read_excel('data/made/discovered.xlsx')

In [31]:
all_other_features = pd.merge(other_features, set_year, on = 'settlement_name_sep', how = 'left')

In [32]:
all_other_features['scheduled_service'] = all_other_features['scheduled_service'].apply(lambda x: 1 if x=='yes' else 0)
all_other_features['type_airport_small'] = all_other_features['type_airport'].apply(lambda x: 1 if x=='small_airport' else 0)

In [34]:
all_other_features.columns

Index(['settlement_name_sep', 'latitude', 'longitude', 'hhru_vacancies_count',
       'POI_num', 'building_num', 'dist_to_airport_geo_km',
       'airport_geo_nearest', 'type_airport', 'scheduled_service',
       'rub_for_life', '1000rub_per_m2_avg', 'sale_house_count', 'set_year',
       'type_airport_small'],
      dtype='object')

In [35]:
all_other_features.to_excel('data/made/all_other_features.xlsx', index = False)